In [15]:
import os
import sys
import spotipy
import spotipy.util as util

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale, minmax_scale

import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import requests
import string

import multiprocessing as mp

import re

from ipywidgets import IntProgress
from IPython.display import display

import nltk

from collections import Counter, defaultdict
from py_genius import Genius
from bs4 import BeautifulSoup

SPOTIPY_REDIRECT_URI='http://localhost:8888/callback'
SPOTIPY_CLIENT_SECRET='c02f97ddd119470d8e24b290722c7490'
SPOTIPY_CLIENT_ID='647ffb8c163b4882be0cafe4bf1352d5'
SCOPE='user-library-read'
USERNAME = '29n1g5zgbzcs413wmovhqdguz'

plt.rcParams['figure.figsize'] = (20, 14)

In [16]:
token = util.prompt_for_user_token(username=USERNAME,
                           scope=SCOPE,
                           client_id=SPOTIPY_CLIENT_ID,
                           client_secret=SPOTIPY_CLIENT_SECRET,
                           redirect_uri=SPOTIPY_REDIRECT_URI)

if not token:
    print("Can't get token for username", username)
    sys.exit(1)

sp = spotipy.Spotify(auth=token)

In [17]:
genius = Genius('D1EklW0p0iQZEo18Ki2SGKBIlBatRKDZvxPcbPYny1Hq2UQo8UJ78abO-DjTXelr')

In [18]:
def get_song_lyrics(song_name):
    search_results = genius.search(song_name)
    try:
        song_url = search_results['response']['hits'][0]['result']['url']
    except:
        return ''
    page = requests.get(song_url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics_bytes = html.find('div', class_='lyrics').get_text().encode('ascii', 'ignore')
    lyrics = lyrics_bytes.decode('ascii')
    
    return re.sub(r'\[(.*?)\]', '', lyrics)

In [19]:
get_song_lyrics('blacker the berry')

'\n\n\n\n\nEverything black, I don\'t want black (They want us to bow)\nI want everything black, I ain\'t need black (Down to our knees)\nSome white, some black, I ain\'t mean black (And pray to a God)\nI want everything black (That we don\'t believe)\nEverything black, want all things black\nI don\'t need black, want everything black\nDon\'t need black, our eyes ain\'t black\nI own black, own everything black\n\n\nSix in the morn\', fire in the street\nBurn, baby, burn, that\'s all I wanna see\nAnd sometimes I get off watchin\' you die in vain\nIt\'s such a shame they may call me crazy\nThey may say I suffer from schizophrenia or somethin\'\nBut homie, you made me\nBlack don\'t crack, my nigga\n\n\nI\'m the biggest hypocrite of 2015\nOnce I finish this, witnesses will convey just what I mean\nBeen feeling this way since I was 16, came to my senses\nYou never liked us anyway, fuck your friendship, I meant it\nI\'m African-American, I\'m African\nI\'m black as the moon, heritage of a sm

In [20]:
def get_lyrics_spotify_uri(uri):
    global sp
    while True:
        try:
            track = sp.track(uri)
            track_name = track['name']
            track_artist = track['artists'][0]['name']
            full_name = '{0} - {1}'.format(track_name, track_artist)
            return get_song_lyrics(full_name)
        except:
            token = util.prompt_for_user_token(username=USERNAME,
                               scope=SCOPE,
                               client_id=SPOTIPY_CLIENT_ID,
                               client_secret=SPOTIPY_CLIENT_SECRET,
                               redirect_uri=SPOTIPY_REDIRECT_URI)

            if not token:
                print("Can't get token for username", username)
                sys.exit(1)

            sp = spotipy.Spotify(auth=token)

In [21]:
test = pickle.load(open('datasets/rap_metafeatures', 'rb'))
# test_dict = {
    
# }
# print(test[0])
# print([x])
testtest = test['uri'][4]
print(get_lyrics_spotify_uri(testtest))
x = get_lyrics_spotify_uri(testtest)




Are you alright?
I'm alright, I'm quite alright
And my money's right
8 (yeah)
Countin' them bands
All way to the top 'til they be fallin' over
(yeah, yeah, yeah)
Countin' them bands
On my way to the top 'til we fallin' over


I don't really care if you cry
On the real, you shoulda never lied
Shoulda saw the way she looked me in my eyes
She said: "Baby, I am not afraid to die."
Push me to the edge
All my friends are dead
Push me to the edge
All my friends are dead
Push me to the edge
All my friends are dead
Push me to the edge


Phantom that's all red, inside all white
Like somethin' you ride a sled down, I just want that head
My Brittany got mad, I'm barely her man now
Everybody got the same swag now
Watch the way that I tear it down
Stackin' my bands all the way to the top
All the way 'til my bands fallin' over
Every time that you leave your spot
Your girlfriend call me like, "Come on over!"
I like the way that she treat me
Gon' leave you, won't leave me, I call it that Casanova
Sh

In [22]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )
        progress.layout.display = 'none'
        box.layout.display = 'none'        

In [23]:
# manager = mp.Manager()
# lyrics_dict = manager.dict()
# output = mp.Queue()

In [24]:
toktok = nltk.tokenize.ToktokTokenizer()

In [25]:
def save_lyrics(genre_matrix, lyrics_d):
    print('begin')
    for uri in log_progress(genre_matrix.uri, every=1):
        if uri not in lyrics_d:
            lyrics = get_lyrics_spotify_uri(uri)
            tokens = set(toktok.tokenize(lyrics.lower()))
            lyrics_d[uri] = tokens
# def save_lyrics(genre_matrix, output):
#     print('begin')
#     lyrics_d = {}
#     for uri in log_progress(genre_matrix.uri, every=1):
#         lyrics = get_lyrics_spotify_uri(uri)
#         tokens = set(toktok.tokenize(lyrics.lower()))
#         lyrics_d[uri] = tokens
#     output.put(lyrics_d)

def get_lyrics_dicts(genre_dict):
    all_classes = list(genre_dict.keys())
    N = 0
    index = defaultdict(dict)
    inverted_index = defaultdict(lambda: defaultdict(set))
        
    for genre_name, genre_matrix in log_progress(genre_dict.items(), every=1):
        N += genre_matrix.shape[0]
        print('Now processing: {}'.format(genre_name))
        for uri in log_progress(genre_matrix.uri, every=1):
            tokens = None
            if uri not in lyrics_dict:
                lyrics = get_lyrics_spotify_uri(uri)
                tokens = set(nltk.tokenize.word_tokenize(lyrics.lower()))
                index[genre_name][uri] = tokens
                lyrics_dict[uri] = tokens
            else:
                index[genre_name][uri] = tokens               
            for token in tokens:
                inverted_index[token][genre_name].add(uri)
    
    return index, inverted_index, N

In [26]:
def mutual_information(t, c, my_index, inverted_index, N):
    # amount of documents that contain tag t, and are in class c
    N_11 =  len(set(inverted_index[t][c])) 

    # amount of documents that contain tag t, but are not in class c
    N_10 =  len(set([
        i for sub in [inverted_index[t][z] for z in [x for x in inverted_index[t].keys() if x != c]]
        for i in sub
    ]))
    
    # amount of documents that do not contain tag t, but are in class c
    N_01 =  len(set([x for x in my_index[c].keys() if t not in my_index[c][x]]))
    
    # amount of documents that do not contain tag t, and are not in class c
    N_00 = N - len(set(my_index[c].keys()) |
                   set([i for sub in [
                       inverted_index[t][s] for s in inverted_index[t].keys() if s != c
                   ] for i in sub]))
    
    # Get all the formula's written down for easier returning
    m_1 = (N_11 / N) * np.log2((N * N_11 / ((N_11 + N_10) * (N_11 + N_01))))\
        if N_11 != 0 and N_11 + N_10 != 0 and N_11 + N_01 != 0 else 0
                
    m_2 = (N_01 / N) * np.log2((N * N_01 / ((N_01 + N_00) * (N_11 + N_01))))\
        if N_01 != 0 and N_01 + N_00 != 0 and N_11 + N_01 != 0 else 0
        
    m_3 = (N_10 / N) * np.log2((N * N_10 / ((N_11 + N_10) * (N_10 + N_00))))\
        if N_10 != 0 and N_11 + N_10 != 0 and N_10 + N_00 != 0 else 0
        
    m_4 = (N_00 / N) * np.log2((N * N_00 / ((N_01 + N_00) * (N_10 + N_00))))\
        if N_00 != 0 and N_01 + N_00 != 0 and N_10 + N_00 != 0 else 0

    return m_1 + m_2 + m_3 + m_4

In [27]:
genre_dict = {
    'rap': pickle.load(open('datasets/rap_metafeatures', 'rb')),
    'techno': pickle.load(open('datasets/techno_metafeatures', 'rb')),
    'soul': pickle.load(open('datasets/soul_metafeatures', 'rb')),
    'grunge': pickle.load(open('datasets/grunge_metafeatures', 'rb')),
    'metal': pickle.load(open('datasets/metal_metafeatures', 'rb')),
    'meditation': pickle.load(open('datasets/meditation_metafeatures', 'rb')),
    'k-pop': pickle.load(open('datasets/k-pop_metafeatures', 'rb')),
    'hardcore': pickle.load(open('datasets/hardcore_metafeatures', 'rb')),
    'rock': pickle.load(open('datasets/rock_metafeatures', 'rb')),
    'punk': pickle.load(open('datasets/punk_metafeatures', 'rb')),
    'jazz': pickle.load(open('datasets/jazz_metafeatures', 'rb')),
    'funk': pickle.load(open('datasets/funk_metafeatures', 'rb')),
    'vaporwave': pickle.load(open('datasets/vaporwave_metafeatures', 'rb')),
    'classical': pickle.load(open('datasets/classical_metafeatures', 'rb')),
    'hiphop': pickle.load(open('datasets/hiphop_metafeatures', 'rb')),
    'house': pickle.load(open('datasets/house_metafeatures', 'rb')),
}

In [28]:
# index, inverted_index, N = get_lyrics_dicts(genre_dict)

# Multiprocessing

In [19]:
output = {}
processes = [mp.Process(target=save_lyrics, args=(genre_matrix[:1000],output)) for genre_matrix in genre_dict.values()]

for p in processes:
    p.start()
    
for p in processes:
    p.join()
    
# results = [output.get() for p in processes]

begin
begin
begin
begin
begin
begin
begin
begin
begin


Widget Javascript not detected.  It may not be installed or enabled properly.
Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


begin


Widget Javascript not detected.  It may not be installed or enabled properly.
Widget Javascript not detected.  It may not be installed or enabled properly.


...

Widget Javascript not detected.  It may not be installed or enabled properly.


retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying .

retrying ...3secs
retrying ...3secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
retrying .

KeyboardInterrupt: 

retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...4secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...2secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...4secs
retrying ...4secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying .

In [ ]:
results

In [ ]:
pickle.dump(results, open('/home/nima/projects/results', 'wb'))

In [22]:
# print(list(lyrics_dict.values())[:3])

In [23]:
pickle.dump(lyrics_dict, open('/home/nima/projects/lyrics_dict', 'wb'))

In [35]:
# len(lyrics_dict)
print(genre_dict['soul'].uri[0])
print(lyrics_dict['spotify:track:3b0EOvScbZUc0qJx0E1L2z'])

spotify:track:3b0EOvScbZUc0qJx0E1L2z


BrokenPipeError: [Errno 32] Broken pipe

In [29]:
lyrics = pd.read_csv('datasets/lyrics.csv')

In [43]:
def get_lyrics_dicts(genre_dict):
    all_classes = list(genre_dict.keys())
    N = 0
    index = defaultdict(dict)
    inverted_index = defaultdict(lambda: defaultdict(set))
        
    for genre_name, lyric_list in genre_dict.items():
        N += len(lyric_list)
        for song, lyric in lyric_list:
            print(song, lyric)
            try:
                tokens = set(toktok.tokenize(lyric.lower()))
            except:
                tokens = set()
            index[genre_name][song] = tokens
            for token in tokens:
                inverted_index[token][genre_name].add(song)
    
    return index, inverted_index, N

In [ ]:
genre_dict = defaultdict(list)

for genre, song, lyric in zip(lyrics.genre, lyrics.song, lyrics.lyrics):
    genre_dict[genre].append((song, lyric))

In [ ]:
# print(lyrics['song'], lyrics['lyrics'])
index, inverted_index, N = get_lyrics_dicts(genre_dict)